## Magentic-One
https://microsoft.github.io/autogen/dev/user-guide/agentchat-user-guide/magentic-one.html

In [ ]:
%pip install "autogen-ext[file-surfer]==0.4.0.dev13"

## Load Azure Configuration

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)


# Helper Method to see the responses per agent

In [3]:
def get_agent_responses(response):
    # ANSI escape code for bold text
    bold_start = "\033[1m"
    bold_end = "\033[0m"

    # ANSI escape code for red text
    red_start = "\033[31m"
    red_end = "\033[0m"

    print(f"{bold_start}****************Agent Responses****************{bold_end}")

    for messages in response.messages:
        source = messages.source
        print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
        print(messages.content)

## Magentic-One with Assistant Agent

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer


travel_agent = AssistantAgent(
    "TravelAgent",
    description="This agent provides a travel plan that includes travel destinations, flights, restaurants, and accommodations.",
    model_client=az_model_client,
    system_message="""
        You are a travel agent that will provide travel plans that in include travel destinations, flights, restaurants, and accommodations.
    """,
)

team = MagenticOneGroupChat([travel_agent], model_client=az_model_client)

task = """
    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.
    Find flights from San Francisco to New York City and get the best deals and format the result in a table
"""
response = await Console(team.run_stream(task=task))



---------- user ----------

    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.
    Find flights from San Francisco to New York City and get the best deals and format the result in a table

---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.
    Find flights from San Francisco to New York City and get the best deals and format the result in a table



To answer this request we have assembled the following team:

TravelAgent: This agent provides a travel plan that includes travel destinations, flights, restaurants, and

In [ ]:
get_agent_responses(response)

## Magentic-One with a MultimodalWebSurfer

In [7]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=az_model_client,
    )

team = MagenticOneGroupChat([surfer], model_client=az_model_client)
response = await Console(team.run_stream(task="Was there a Lakers game on January 5, 2025? Who won that game?"))

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:149: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(planning_conversation, cancellation_token=ctx.cancellation_token)
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:160: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(planning_conversation, cancellation_token=ctx.cancellation_token)


---------- user ----------
Was there a Lakers game on January 5, 2025? Who won that game?
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

Was there a Lakers game on January 5, 2025? Who won that game?


To answer this request we have assembled the following team:

WebSurfer: 
    A helpful assistant with access to a web browser.
    Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, etc., filling in form fields, etc.).
    It can also summarize the entire page, or answer questions based on the content of the page.
    It can also be asked to sleep and wait for pages to load, in cases where the pages seem to be taking a while to load.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
- The request involves the Los Angeles Lakers.
- The specific date in question is January 5, 2025.

2. FACTS TO LOOK UP
- Whether the Los Angeles Lakers had a game

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:412: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["vision"] is False:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:427: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["json_output"] is False and json_output is True:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:433: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["function_calling"] is False and len(tools) > 0:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:503: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.

---------- WebSurfer ----------
I typed 'Lakers game January 5, 2025 result' into '0 characters out of 2000'.

 Here is a screenshot of the webpage: [Lakers game January 5, 2025 result - Search](https://www.bing.com/search?q=Lakers+game+January+5%2C+2025+result&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=C386A6379E57435E86701E49175D728B).
 The viewport shows 32% of the webpage, and is positioned at the top of the page 
The following metadata was extracted from the webpage:

{
    "meta_tags": {
        "referrer": "origin-when-cross-origin",
        "SystemEntropyOriginTrialToken": "A5is4nwJJVnhaJpUr1URgj4vvAXSiHoK0VBbM9fawMskbDUj9WUREpa3JzGAo6xd1Cp2voQEG1h6NQ71AsMznU8AAABxeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiTXNVc2VyQWdlbnRMYXVuY2hOYXZUeXBlIiwiZXhwaXJ5IjoxNzUzNzQ3MjAwLCJpc1N1YmRvbWFpbiI6dHJ1ZX0=",
        "ConfidenceOriginTrialToken": "Aqw360MHzRcmtEVv55zzdIWcTk2BBYHcdBAOysNJZP4qkN8M+5vUq36ITHFVst8LiX36KBZJXB8xvyBgdK2z5Q0AAAB6eyJvcmlnaW4iOiJodHRwczovL2Jpbm

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:419: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["json_output"] is False and json_output is True:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:284: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, json_output=True)


---------- MagenticOneOrchestrator ----------
Yes, there was a Los Angeles Lakers game on January 5, 2025. The Lakers played against the Houston Rockets and the Rockets won the game with a score of 119-115.
---------- Summary ----------
Number of messages: 5
Finish reason: The search results indicate that there was a Lakers game on January 5, 2025, against the Houston Rockets, and the Houston Rockets won the game 119-115.
Total prompt tokens: 3144
Total completion tokens: 67
Duration: 24.00 seconds


/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:400: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)


In [8]:
get_agent_responses(response)

****************Agent Responses****************
user
Was there a Lakers game on January 5, 2025? Who won that game?
MagenticOneOrchestrator

We are working to address the following user request:

Was there a Lakers game on January 5, 2025? Who won that game?


To answer this request we have assembled the following team:

WebSurfer: 
    A helpful assistant with access to a web browser.
    Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, etc., filling in form fields, etc.).
    It can also summarize the entire page, or answer questions based on the content of the page.
    It can also be asked to sleep and wait for pages to load, in cases where the pages seem to be taking a while to load.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
- The request involves the Los Angeles Lakers.
- The specific date in question is January 5, 2025.

2. FACTS TO LOOK UP
- Whether the Los Angeles Lakers had a 

## Magentic-One with a Code Executor Agent

In [9]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

# Create a code executor agent that uses a Docker container to execute code.
code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
await code_executor.start()
code_executor_agent = CodeExecutorAgent("code_executor", code_executor=code_executor)

team = MagenticOneGroupChat([code_executor_agent], model_client=az_model_client, max_turns=50)
task = f"""
You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar

"""
response = await Console(team.run_stream(task=task))

# Stop the code executor.
await code_executor.stop()

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:149: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(planning_conversation, cancellation_token=ctx.cancellation_token)
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:160: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(planning_conversation, cancellation_token=ctx.cancellation_token)


---------- user ----------

You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar


---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar




To answer this request we have assembled the following team:

code_executor: A computer terminal that performs no other actio

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:377: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)


---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar




To answer this request we have assembled the following team:

code_executor: A computer terminal that performs no other action than running Python scripts (provided to it quoted in ```python code blocks), or sh shell scripts (provided to it quoted in ```sh code blocks).


Here is an initial fact sheet to consider:

Here is the revised fact sheet with updates:

1. **GIVEN OR VERIFIED FACTS**
   - Dataset name: bigfootsightings.csv
   - Each row represents 1 sighting
   - The top 10 years with the most sightings need to 

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:387: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)


---------- MagenticOneOrchestrator ----------
Run the following shell command to install pandas and matplotlib:
```sh
pip install pandas matplotlib
```
Once installed, execute the following Python script:
```python
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('bigfootsightings.csv')

# Extract the year from the date column
df['year'] = pd.to_datetime(df['date'], errors='coerce').dt.year

# Count the number of sightings per year
sightings_per_year = df['year'].value_counts().sort_index()

# Identify the top 10 years with the most sightings
top_10_years = sightings_per_year.nlargest(10).sort_values(ascending=False)

# Create a column chart
plt.figure(figsize=(10, 6))
bar_plot = top_10_years.plot(kind='bar', color='skyblue')
plt.title('Top 10 Years with the Most Bigfoot Sightings')
plt.xlabel('Year')
plt.ylabel('Number of Sightings')
plt.xticks(rotation=45)

# Annotate the chart with the sighting values on top of each bar
for i in bar_plot.conta

In [ ]:
get_agent_responses(response)


## Magentic-One with File Surfer

In [ ]:
from autogen_ext.agents.file_surfer._file_surfer import FileSurfer
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console

file_path = "../Data/products/"
# Create a file surfer agent.
file_surfer = FileSurfer(
    "FileSurfer",
    model_client=az_model_client
)
file_surfer._browser.open_path(file_path)


team = MagenticOneGroupChat(
    [file_surfer],
    model_client=az_model_client, 
    max_turns=50,)

task = """What is the price of the TrailMaster X4 Tent and what are its features?"""
response = await Console(team.run_stream(task=task))


/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:149: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(planning_conversation, cancellation_token=ctx.cancellation_token)
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:160: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(planning_conversation, cancellation_token=ctx.cancellation_token)
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:284: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response

---------- user ----------
What is the price of the TrailMaster X4 Tent and what are its features?
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

What is the price of the TrailMaster X4 Tent and what are its features?


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
None provided in the request.

2. FACTS TO LOOK UP
- The price of the TrailMaster X4 Tent
- The features of the TrailMaster X4 Tent
These could be found on the official TrailMaster website, on popular outdoor gear retailers' websites (such as REI, Amazon, or Cabela's), or through gear review sites and forums.

3. FACTS TO DERIVE
None specifically required as the request is for looking up specific information.

4. EDUCATED GUESSES
None, as the requested information should be directly looked up from authoritative sources.


Here is the 

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/agents/file_surfer/_file_surfer.py:128: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  create_result = await self._model_client.create(


---------- FileSurfer ----------
Path: /workspaces/AzureAI/Data/products
Viewport position: Showing page 1 of 1.

# Index of /workspaces/AzureAI/Data/products

| Name | Size | Date Modified |
| ---- | ---- | ------------- |
| .. (parent directory) | | |
| product_info_19.pdf | 136024 | 2024-12-12 17:21 |
| product_info_13.pdf | 133012 | 2024-12-12 17:21 |
| product_info_9.pdf | 136807 | 2024-12-12 17:21 |
| product_info_7.pdf | 130423 | 2024-12-12 17:21 |
| product_info_10.pdf | 134498 | 2024-12-12 17:21 |
| product_info_3.pdf | 135879 | 2024-12-12 17:21 |
| product_info_8.pdf | 134844 | 2024-12-12 17:21 |
| product_info_4.pdf | 77127 | 2024-12-12 17:21 |
| product_info_14.pdf | 132788 | 2024-12-12 17:21 |
| product_info_20.pdf | 136219 | 2024-12-12 17:21 |
| product_info_17.pdf | 136838 | 2024-12-12 17:21 |
| product_info_1.pdf | 134822 | 2024-12-12 17:21 |
| product_info_5.pdf | 136933 | 2024-12-12 17:21 |
| product_info_6.pdf | 136851 | 2024-12-12 17:21 |
| product_info_16.pdf | 139

/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:380: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:390: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)


In [ ]:
get_agent_responses(response)

## Analyzing Companies

In [14]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console

# Create a code executor agent that uses a Docker container to execute code.
code_executor = DockerCommandLineCodeExecutor(work_dir="finance")
await code_executor.start()
code_executor_agent = CodeExecutorAgent("code_executor", code_executor=code_executor)

finance_agent = AssistantAgent(
    "FinanceAgent",
    description="This agent provides analysis of stock prices and financial data.",
    model_client=az_model_client,
    system_message="""
        You are a finance agent that will provide analysis of stock prices and financial data.
        You will use the code executor to get the data and do the analysis.
    """,
)

task = """
    Create a comparison of the stock prices of NVIDIA, Tesla, and Microsoft over the last 3 years.
    Use the Yahoo Finance API to get the data and do the analysis.
    Create a line chart with the stock prices in the Y-axis and the date in the X-axis and save it to a PNG file.
    Use the code executor to get the data and do the analysis.
    Create a column chart to compare their revenue and profit over the last 3 years.
"""

team = MagenticOneGroupChat([code_executor_agent, finance_agent], model_client=az_model_client)
response = await Console(team.run_stream(task=task))

# Stop the code executor.
await code_executor.stop()

---------- user ----------

    Create a comparison of the stock prices of NVIDIA, Tesla, and Microsoft over the last 3 years.
    Use the Yahoo Finance API to get the data and do the analysis.
    Create a line chart with the stock prices in the Y-axis and the date in the X-axis and save it to a PNG file.
    Use the code executor to get the data and do the analysis.
    Create a column chart to compare their revenue and profit over the last 3 years.

---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


    Create a comparison of the stock prices of NVIDIA, Tesla, and Microsoft over the last 3 years.
    Use the Yahoo Finance API to get the data and do the analysis.
    Create a line chart with the stock prices in the Y-axis and the date in the X-axis and save it to a PNG file.
    Use the code executor to get the data and do the analysis.
    Create a column chart to compare their revenue and profit over the last 3 years.



To answer

In [13]:
get_agent_responses(response)

****************Agent Responses****************
user

    Create a comparison of the stock prices of NVIDIA, Tesla, and Microsoft over the last 3 years.
    Use the Yahoo Finance API to get the data and do the analysis.
    Create a line chart with the stock prices in the Y-axis and the date in the X-axis and save it to a PNG file.
    Put the value on top of each line.
    Use the code executor to get the data and do the analysis.
    Create a table to analyze their revenue and profit over the last 3 years.

MagenticOneOrchestrator

We are working to address the following user request:


    Create a comparison of the stock prices of NVIDIA, Tesla, and Microsoft over the last 3 years.
    Use the Yahoo Finance API to get the data and do the analysis.
    Create a line chart with the stock prices in the Y-axis and the date in the X-axis and save it to a PNG file.
    Put the value on top of each line.
    Use the code executor to get the data and do the analysis.
    Create a table to 